In [1]:
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
sys.path.append('..\src')
sys.path.append("src")

In [2]:
from methods.analytic.exact import AnalyticalMathodOld

In [3]:
method = AnalyticalMathodOld()


In [4]:
# read csv with steel images
all_steel_df = pd.read_csv("../data/generated/fourier/steel/parameters.csv", index_col=False)
avg_steel_df = pd.read_csv("../data/generated/fourier/single/parameters.csv", index_col=False)

# add prefix "all/steel/" to the filename column
all_steel_df['filename'] = "fourier/steel/" + all_steel_df['filename'].astype(str)
# the same for the average dataframe
avg_steel_df['filename'] = "fourier/single/" + avg_steel_df['filename'].astype(str)

main_dir = "../data/generated/"

all_steel_train, all_steel_test = train_test_split(all_steel_df, test_size=0.4, random_state=12, shuffle=True, stratify=all_steel_df['epsilon'])
all_steel_test, all_steel_valid = train_test_split(all_steel_test, test_size=1000, random_state=12, shuffle=True, stratify=all_steel_test['epsilon'])

avg_steel_train, avg_steel_test = train_test_split(avg_steel_df, test_size=0.4, random_state=12, shuffle=True, stratify=avg_steel_df['epsilon'])
avg_steel_test, avg_steel_valid = train_test_split(avg_steel_test, test_size=1000, random_state=12, shuffle=True, stratify=avg_steel_test['epsilon'])

In [4]:
import os

# Directory to scan for PNG files
directory_path = '../data/raw/steel/1channel'

# List to store PNG filenames
png_filenames = []

# Scan the directory for PNG files
for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith('.png'):
            png_filenames.append(file)

len(png_filenames)

100000

In [6]:
# avg_steel_test
# iterate over the dataframe, read the image into numpy array using cv2 and run method.calculate_epsilon()
# save the result to the list. First image need to be loaded, function takes numpy array as input

main_dir = "../data/raw/steel/1channel/"

results = []
from tqdm import tqdm
import cv2
for i in tqdm(png_filenames):
    img = cv2.imread(main_dir + i, cv2.IMREAD_GRAYSCALE)
    epsilon = method.calculate_epsilon(img)
    results.append(epsilon[0])


  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [15:23<00:00, 108.33it/s]


In [7]:
df = pd.DataFrame({'filename': png_filenames, 'epsilon': results})

df.to_csv("../data/results/exact_method/exact_method-raw_steel_predictions.csv", index=False)

In [6]:
def cyclic_MAE_test(output, target):
    return min(abs(output-target), min(abs(output-target+1), abs(output-target-1)))

In [7]:
def save_results(outputs, targets, filename: str):
    differences = []
    for out, tar in zip(outputs, targets):
        differences.append(cyclic_MAE_test(out, tar))
    results_df = pd.DataFrame(list(zip(targets, outputs, differences)),
                  columns =['epsilon', 'predicted', 'loss'])
    results_df.to_csv(filename, index=False)
    return results_df

In [10]:
save_results(results, all_steel_test['epsilon'], "exact_method-steel_fourier_results.csv")

,epsilon,predicted,loss
0,0.426,0.432225,0.006225
1,0.033,0.026574,0.006426
2,0.487,0.503295,0.016295
3,0.489,0.448769,0.040231
4,0.335,0.303809,0.031191
...,...,...,...
2995,0.267,0.303114,0.036114
2996,0.457,0.434686,0.022314
2997,0.042,0.028159,0.013841
2998,0.903,0.909469,0.006469
